# Variant Annotation with Hail: Annotation Database

This notebook shows how to annotate genomic data in a Hail MatrixTable with Hail's Annotation Database ("a curated collection of variant annotations in an accessible and Hail-friendly format") and save it as a Hail Table to an Apollo database (dnax://) on the DNAnexus platform. See documentation for guidance on launch specs for the JupyterLab with Spark Cluster app for different data sizes: https://documentation.dnanexus.com/science/using-hail-to-analyze-genomic-data

Note: Functionality around Hail's Annotation Database is experimental and subject to change.

Additional documentation: https://hail.is/docs/0.2/annotation_database_ui.html#id1

Pre-conditions for running this notebook successfully:
- There is an existing Hail MatrixTable in DNAX

## 1) Initiate Spark and Hail

In [ ]:
# Running this cell will output a red-colored message- this is expected.
# The 'Welcome to Hail' message in the output will indicate that Hail is ready to use in the notebook.

from pyspark.sql import SparkSession
import hail as hl

builder = (
    SparkSession
    .builder
    .enableHiveSupport()
)
spark = builder.getOrCreate()
hl.init(sc=spark.sparkContext)

## 2) Read MT

In [ ]:
# define MT url

mt_url = "dnax://database-GFpXJ5j0vzZxPZQ2Ggf14x7q/geno.mt"

In [ ]:
# read MT

mt = hl.read_matrix_table(mt_url)

In [ ]:
# View structure of MT before annotation

mt.describe()

## 3) Create Hail Annotation Database

Let's create an instance of a Database so we can get access to Hail's curated collection of variant annotations.

Note: Hail's collection of variant annotations are stored in AWS (S3) and only available in a bucket in the US region.

*Additional documentation: https://hail.is/docs/0.2/annotation_database_ui.html#id1*

In [ ]:
# Create Database

db = hl.experimental.DB(region='us', cloud='aws')

In [ ]:
# Run to view available datasets
# db.available_datasets

## 4) Annotate

Let's annotate our MT with the selected dataset: 'gencode'

*Additional documentation: https://hail.is/docs/0.2/experimental/hail.experimental.DB.html#hail.experimental.DB.annotate_rows_db*

In [ ]:
# Annotate MT

ann_mt = db.annotate_rows_db(mt, 'gencode')

In [ ]:
# See details of MT after annotation

ann_mt.describe()

We see that `gencode` has been added to the row fields of the MT

## 5) Create Hail DB Annotated Table and save in Apollo Database

In [ ]:
# Create Hail Table from MT

ann_tb = ann_mt.rows()

In [ ]:
# Define database and table name

db_name = "database_name"
tb_name = "ann_haildb.ht"

In [ ]:
# Create database in DNAX

stmt = f"CREATE DATABASE IF NOT EXISTS {db_name} LOCATION 'dnax://'"
print(stmt)
spark.sql(stmt).show()

In [ ]:
# Store Table in DNAX

import dxpy

# find database ID of newly created database using a dxpy method
db_uri = dxpy.find_one_data_object(name=f"{db_name}", classname="database")['id']
url = f"dnax://{db_uri}/{tb_name}"

# Note: Writing (saving/storing) the Table to the database can be computationally expensive 
# depending on the size of the annotations. 
# 
# Before this step, the Hail Table is just an object in memory. To persist it and be able to access 
# it later, the notebook needs to write it into a persistent filesystem (in this case DNAX).
# See https://hail.is/docs/0.2/hail.Table.html#hail.Table.write for additional documentation.
ann_tb.write(url) # output should describe size of Table (i.e. number of rows, partitions)